In [1]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm 
import pprint as pp

In [2]:
TRAIN_DIR = 'train'#'X:/Kaggle_Data/dogs_vs_cats/train/train'
TEST_DIR = 'test'#'X:/Kaggle_Data/dogs_vs_cats/test/test'
IMG_SIZE = 64
LR = 1e-3
train_path = "/Users/jian/Desktop/CNN-TinyImagenet/tiny-imagenet-200/train"
label_path = "/Users/jian/Desktop/CNN-TinyImagenet/tiny-imagenet-200"

In [3]:
train_data =[]
label = []
counter = 0
for file in tqdm(os.listdir(train_path)):
    if counter == 20:
        break
    for pic in os.listdir(train_path+"/"+file+"/images"):
        img = cv2.imread(train_path+"/"+file+"/images/"+pic)
        train_data.append(np.array(img))
        label.append(file)
    counter += 1
train_data = np.array(train_data)
# np.save('tiny-imagenet-train.npy', train_data)

 10%|█         | 20/200 [00:04<00:33,  5.38it/s]


In [4]:
label = np.array(label)

label = np.reshape(label, (len(label), 1))


In [5]:
lab_id = {}
lab = []
seen = []
labels = []
counter = 0
for l in tqdm(range(label.shape[0])):
    if counter == 20:
        break
    if label[l] in seen:
        continue
    else:
        seen.append(label[l])
        l_id = str(label[l])
        new_dict_entry = {l_id : counter}
        counter += 1 
        lab_id.update(new_dict_entry)
# print(lab_id) 
num_classes = counter
print(num_classes)

  0%|          | 0/10000 [00:00<?, ?it/s]

20


In [6]:
for l in tqdm(range(label.shape[0])):
    temp = np.zeros(num_classes)
    clss = label[l]
    index = lab_id.get(str(clss))
    temp[index] = 1
    labels.append(temp)

100%|██████████| 10000/10000 [00:00<00:00, 52374.66it/s]


In [7]:
labels = np.array(labels)
# print(labels.shape)

In [8]:
training_data = []
for l in tqdm(range(label.shape[0])):
    training_data.append([train_data[l], labels[l]])
    
shuffle(training_data)

np.save('training_data.npy', training_data)


100%|██████████| 10000/10000 [00:00<00:00, 1027109.41it/s]


In [9]:
import data_augmentation as da
DA_FACTOR = 2
trn_data = np.load('training_data.npy')
print(trn_data.shape)
print(trn_data[0].shape)
# print(trn_data[0])

ag_trn_data = np.empty([DA_FACTOR,2])
# print(trn_data[0])
for img in trn_data:
    new_data = np.empty([DA_FACTOR,2])
    new_data = da.data_aug(img)
    print('new_data',new_data.shape)
    print('ag_train_data before concate',ag_trn_data.shape)
    ag_trn_data = np.concatenate([ag_trn_data, new_data])
print ('date augmented successfully', ag_trn_data.shape)
del_indices = range(DA_FACTOR)
print('del indices',del_indices)
ag_trn_data = np.delete(ag_trn_data, del_indices, 0)
print('newx shape',ag_trn_data.shape)
##todo:check new_x[2] should equal to new_data(X[0])
trn_data = ag_trn_data


(10000, 2)
(2,)
new_data (2, 2)
ag_train_data before concate (2, 2)
new_data (2, 2)
ag_train_data before concate (4, 2)
new_data (2, 2)
ag_train_data before concate (6, 2)
new_data (2, 2)
ag_train_data before concate (8, 2)
new_data (2, 2)
ag_train_data before concate (10, 2)
new_data (2, 2)
ag_train_data before concate (12, 2)
new_data (2, 2)
ag_train_data before concate (14, 2)
new_data (2, 2)
ag_train_data before concate (16, 2)
new_data (2, 2)
ag_train_data before concate (18, 2)
new_data (2, 2)
ag_train_data before concate (20, 2)
new_data (2, 2)
ag_train_data before concate (22, 2)
new_data (2, 2)
ag_train_data before concate (24, 2)
new_data (2, 2)
ag_train_data before concate (26, 2)
new_data (2, 2)
ag_train_data before concate (28, 2)
new_data (2, 2)
ag_train_data before concate (30, 2)
new_data (2, 2)
ag_train_data before concate (32, 2)
new_data (2, 2)
ag_train_data before concate (34, 2)
new_data (2, 2)
ag_train_data before concate (36, 2)
new_data (2, 2)
ag_train_data befo

new_data (2, 2)
ag_train_data before concate (928, 2)
new_data (2, 2)
ag_train_data before concate (930, 2)
new_data (2, 2)
ag_train_data before concate (932, 2)
new_data (2, 2)
ag_train_data before concate (934, 2)
new_data (2, 2)
ag_train_data before concate (936, 2)
new_data (2, 2)
ag_train_data before concate (938, 2)
new_data (2, 2)
ag_train_data before concate (940, 2)
new_data (2, 2)
ag_train_data before concate (942, 2)
new_data (2, 2)
ag_train_data before concate (944, 2)
new_data (2, 2)
ag_train_data before concate (946, 2)
new_data (2, 2)
ag_train_data before concate (948, 2)
new_data (2, 2)
ag_train_data before concate (950, 2)
new_data (2, 2)
ag_train_data before concate (952, 2)
new_data (2, 2)
ag_train_data before concate (954, 2)
new_data (2, 2)
ag_train_data before concate (956, 2)
new_data (2, 2)
ag_train_data before concate (958, 2)
new_data (2, 2)
ag_train_data before concate (960, 2)
new_data (2, 2)
ag_train_data before concate (962, 2)
new_data (2, 2)
ag_train_dat

new_data (2, 2)
ag_train_data before concate (1824, 2)
new_data (2, 2)
ag_train_data before concate (1826, 2)
new_data (2, 2)
ag_train_data before concate (1828, 2)
new_data (2, 2)
ag_train_data before concate (1830, 2)
new_data (2, 2)
ag_train_data before concate (1832, 2)
new_data (2, 2)
ag_train_data before concate (1834, 2)
new_data (2, 2)
ag_train_data before concate (1836, 2)
new_data (2, 2)
ag_train_data before concate (1838, 2)
new_data (2, 2)
ag_train_data before concate (1840, 2)
new_data (2, 2)
ag_train_data before concate (1842, 2)
new_data (2, 2)
ag_train_data before concate (1844, 2)
new_data (2, 2)
ag_train_data before concate (1846, 2)
new_data (2, 2)
ag_train_data before concate (1848, 2)
new_data (2, 2)
ag_train_data before concate (1850, 2)
new_data (2, 2)
ag_train_data before concate (1852, 2)
new_data (2, 2)
ag_train_data before concate (1854, 2)
new_data (2, 2)
ag_train_data before concate (1856, 2)
new_data (2, 2)
ag_train_data before concate (1858, 2)
new_data (

new_data (2, 2)
ag_train_data before concate (2744, 2)
new_data (2, 2)
ag_train_data before concate (2746, 2)
new_data (2, 2)
ag_train_data before concate (2748, 2)
new_data (2, 2)
ag_train_data before concate (2750, 2)
new_data (2, 2)
ag_train_data before concate (2752, 2)
new_data (2, 2)
ag_train_data before concate (2754, 2)
new_data (2, 2)
ag_train_data before concate (2756, 2)
new_data (2, 2)
ag_train_data before concate (2758, 2)
new_data (2, 2)
ag_train_data before concate (2760, 2)
new_data (2, 2)
ag_train_data before concate (2762, 2)
new_data (2, 2)
ag_train_data before concate (2764, 2)
new_data (2, 2)
ag_train_data before concate (2766, 2)
new_data (2, 2)
ag_train_data before concate (2768, 2)
new_data (2, 2)
ag_train_data before concate (2770, 2)
new_data (2, 2)
ag_train_data before concate (2772, 2)
new_data (2, 2)
ag_train_data before concate (2774, 2)
new_data (2, 2)
ag_train_data before concate (2776, 2)
new_data (2, 2)
ag_train_data before concate (2778, 2)
new_data (

new_data (2, 2)
ag_train_data before concate (3646, 2)
new_data (2, 2)
ag_train_data before concate (3648, 2)
new_data (2, 2)
ag_train_data before concate (3650, 2)
new_data (2, 2)
ag_train_data before concate (3652, 2)
new_data (2, 2)
ag_train_data before concate (3654, 2)
new_data (2, 2)
ag_train_data before concate (3656, 2)
new_data (2, 2)
ag_train_data before concate (3658, 2)
new_data (2, 2)
ag_train_data before concate (3660, 2)
new_data (2, 2)
ag_train_data before concate (3662, 2)
new_data (2, 2)
ag_train_data before concate (3664, 2)
new_data (2, 2)
ag_train_data before concate (3666, 2)
new_data (2, 2)
ag_train_data before concate (3668, 2)
new_data (2, 2)
ag_train_data before concate (3670, 2)
new_data (2, 2)
ag_train_data before concate (3672, 2)
new_data (2, 2)
ag_train_data before concate (3674, 2)
new_data (2, 2)
ag_train_data before concate (3676, 2)
new_data (2, 2)
ag_train_data before concate (3678, 2)
new_data (2, 2)
ag_train_data before concate (3680, 2)
new_data (

...

new_data (2, 2)
ag_train_data before concate (17126, 2)
new_data (2, 2)
ag_train_data before concate (17128, 2)
new_data (2, 2)
ag_train_data before concate (17130, 2)
new_data (2, 2)
ag_train_data before concate (17132, 2)
new_data (2, 2)
ag_train_data before concate (17134, 2)
new_data (2, 2)
ag_train_data before concate (17136, 2)
new_data (2, 2)
ag_train_data before concate (17138, 2)
new_data (2, 2)
ag_train_data before concate (17140, 2)
new_data (2, 2)
ag_train_data before concate (17142, 2)
new_data (2, 2)
ag_train_data before concate (17144, 2)
new_data (2, 2)
ag_train_data before concate (17146, 2)
new_data (2, 2)
ag_train_data before concate (17148, 2)
new_data (2, 2)
ag_train_data before concate (17150, 2)
new_data (2, 2)
ag_train_data before concate (17152, 2)
new_data (2, 2)
ag_train_data before concate (17154, 2)
new_data (2, 2)
ag_train_data before concate (17156, 2)
new_data (2, 2)
ag_train_data before concate (17158, 2)
new_data (2, 2)
ag_train_data before concate (17

new_data (2, 2)
ag_train_data before concate (17840, 2)
new_data (2, 2)
ag_train_data before concate (17842, 2)
new_data (2, 2)
ag_train_data before concate (17844, 2)
new_data (2, 2)
ag_train_data before concate (17846, 2)
new_data (2, 2)
ag_train_data before concate (17848, 2)
new_data (2, 2)
ag_train_data before concate (17850, 2)
new_data (2, 2)
ag_train_data before concate (17852, 2)
new_data (2, 2)
ag_train_data before concate (17854, 2)
new_data (2, 2)
ag_train_data before concate (17856, 2)
new_data (2, 2)
ag_train_data before concate (17858, 2)
new_data (2, 2)
ag_train_data before concate (17860, 2)
new_data (2, 2)
ag_train_data before concate (17862, 2)
new_data (2, 2)
ag_train_data before concate (17864, 2)
new_data (2, 2)
ag_train_data before concate (17866, 2)
new_data (2, 2)
ag_train_data before concate (17868, 2)
new_data (2, 2)
ag_train_data before concate (17870, 2)
new_data (2, 2)
ag_train_data before concate (17872, 2)
new_data (2, 2)
ag_train_data before concate (17

new_data (2, 2)
ag_train_data before concate (18536, 2)
new_data (2, 2)
ag_train_data before concate (18538, 2)
new_data (2, 2)
ag_train_data before concate (18540, 2)
new_data (2, 2)
ag_train_data before concate (18542, 2)
new_data (2, 2)
ag_train_data before concate (18544, 2)
new_data (2, 2)
ag_train_data before concate (18546, 2)
new_data (2, 2)
ag_train_data before concate (18548, 2)
new_data (2, 2)
ag_train_data before concate (18550, 2)
new_data (2, 2)
ag_train_data before concate (18552, 2)
new_data (2, 2)
ag_train_data before concate (18554, 2)
new_data (2, 2)
ag_train_data before concate (18556, 2)
new_data (2, 2)
ag_train_data before concate (18558, 2)
new_data (2, 2)
ag_train_data before concate (18560, 2)
new_data (2, 2)
ag_train_data before concate (18562, 2)
new_data (2, 2)
ag_train_data before concate (18564, 2)
new_data (2, 2)
ag_train_data before concate (18566, 2)
new_data (2, 2)
ag_train_data before concate (18568, 2)
new_data (2, 2)
ag_train_data before concate (18

new_data (2, 2)
ag_train_data before concate (19200, 2)
new_data (2, 2)
ag_train_data before concate (19202, 2)
new_data (2, 2)
ag_train_data before concate (19204, 2)
new_data (2, 2)
ag_train_data before concate (19206, 2)
new_data (2, 2)
ag_train_data before concate (19208, 2)
new_data (2, 2)
ag_train_data before concate (19210, 2)
new_data (2, 2)
ag_train_data before concate (19212, 2)
new_data (2, 2)
ag_train_data before concate (19214, 2)
new_data (2, 2)
ag_train_data before concate (19216, 2)
new_data (2, 2)
ag_train_data before concate (19218, 2)
new_data (2, 2)
ag_train_data before concate (19220, 2)
new_data (2, 2)
ag_train_data before concate (19222, 2)
new_data (2, 2)
ag_train_data before concate (19224, 2)
new_data (2, 2)
ag_train_data before concate (19226, 2)
new_data (2, 2)
ag_train_data before concate (19228, 2)
new_data (2, 2)
ag_train_data before concate (19230, 2)
new_data (2, 2)
ag_train_data before concate (19232, 2)
new_data (2, 2)
ag_train_data before concate (19

new_data (2, 2)
ag_train_data before concate (19850, 2)
new_data (2, 2)
ag_train_data before concate (19852, 2)
new_data (2, 2)
ag_train_data before concate (19854, 2)
new_data (2, 2)
ag_train_data before concate (19856, 2)
new_data (2, 2)
ag_train_data before concate (19858, 2)
new_data (2, 2)
ag_train_data before concate (19860, 2)
new_data (2, 2)
ag_train_data before concate (19862, 2)
new_data (2, 2)
ag_train_data before concate (19864, 2)
new_data (2, 2)
ag_train_data before concate (19866, 2)
new_data (2, 2)
ag_train_data before concate (19868, 2)
new_data (2, 2)
ag_train_data before concate (19870, 2)
new_data (2, 2)
ag_train_data before concate (19872, 2)
new_data (2, 2)
ag_train_data before concate (19874, 2)
new_data (2, 2)
ag_train_data before concate (19876, 2)
new_data (2, 2)
ag_train_data before concate (19878, 2)
new_data (2, 2)
ag_train_data before concate (19880, 2)
new_data (2, 2)
ag_train_data before concate (19882, 2)
new_data (2, 2)
ag_train_data before concate (19

In [10]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')
#       net = layers.conv2d(inputs, 64, [11, 11], 4, padding='VALID', scope='conv1')
#       net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool1')

#       net = layers.conv2d(net, 192, [5, 5], scope='conv2')
#       net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool2')

#       net = layers.conv2d(net, 384, [3, 3], scope='conv3')
#       net = layers.conv2d(net, 384, [3, 3], scope='conv4')
#       net = layers.conv2d(net, 256, [3, 3], scope='conv5')
#       net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool5')
convnet = conv_2d(convnet, 64, [11,11],  4, activation='relu')
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = conv_2d(convnet, 192, [5,5], activation='relu')
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = conv_2d(convnet, 384, [3,3], activation='relu')

convnet = conv_2d(convnet, 384, [3,3], activation='relu')

convnet = conv_2d(convnet, 256, [3,3], activation='relu')
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = fully_connected(convnet, 1024, activation='relu')

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, num_classes, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log',max_checkpoints=1, tensorboard_verbose=0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [11]:
# print(trn_data)
train = trn_data[:-500]
test = trn_data[-500:]

In [12]:
x = np.array([i[0] for i in train])
# print(x.shape)
y = [i[1] for i in train]
# print(y[0])

test_x = np.array([i[0] for i in test])
test_y = [i[1] for i in test]

In [13]:
MODEL_NAME = 'test_cnn-{}-{}.model'.format(LR, '2conv-basic')
model.fit({'input': x}, {'targets': y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}),
          show_metric=True, snapshot_step=1, snapshot_epoch=True, run_id=MODEL_NAME)



---------------------------------
Run id: test_cnn-0.001-2conv-basic.model
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 19500
Validation samples: 500
--
Training Step: 1  | time: 1.546s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 | val_loss: 19.54685 - val_acc: 0.0560 -- iter: 00064/19500
--
Training Step: 2  | total loss: 5.24011 | time: 2.796s
| Adam | epoch: 001 | loss: 5.24011 - acc: 0.0000 | val_loss: 20.78814 - val_acc: 0.0840 -- iter: 00128/19500
--
Training Step: 3  | total loss: 17.42965 | time: 4.061s
| Adam | epoch: 001 | loss: 17.42965 - acc: 0.0639 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 00192/19500
--
Training Step: 4  | total loss: 20.13761 | time: 5.314s
| Adam | epoch: 001 | loss: 20.13761 - acc: 0.0511 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 00256/19500
--
Training Step: 5  | total loss: 21.88808 | time: 6.565s
| Adam | epoch: 001 | 

Training Step: 44  | total loss: 21.98804 | time: 55.687s
| Adam | epoch: 001 | loss: 21.98804 - acc: 0.0451 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 02816/19500
--
Training Step: 45  | total loss: 21.98099 | time: 56.944s
| Adam | epoch: 001 | loss: 21.98099 - acc: 0.0454 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 02880/19500
--
Training Step: 46  | total loss: 21.97524 | time: 58.209s
| Adam | epoch: 001 | loss: 21.97524 - acc: 0.0456 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 02944/19500
--
Training Step: 47  | total loss: 21.91167 | time: 59.472s
| Adam | epoch: 001 | loss: 21.91167 - acc: 0.0484 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 03008/19500
--
Training Step: 48  | total loss: 21.91727 | time: 60.734s
| Adam | epoch: 001 | loss: 21.91727 - acc: 0.0481 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 03072/19500
--
Training Step: 49  | total loss: 21.86508 | time: 62.001s
| Adam | epoch: 001 | loss: 21.86508 - acc: 0.0504 | val_loss: 22.10482 - val_acc:

Training Step: 88  | total loss: 21.87051 | time: 111.073s
| Adam | epoch: 001 | loss: 21.87051 - acc: 0.0502 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 05632/19500
--
Training Step: 89  | total loss: 21.95007 | time: 112.348s
| Adam | epoch: 001 | loss: 21.95007 - acc: 0.0467 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 05696/19500
--
Training Step: 90  | total loss: 21.91373 | time: 113.601s
| Adam | epoch: 001 | loss: 21.91373 - acc: 0.0483 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 05760/19500
--
Training Step: 91  | total loss: 21.98897 | time: 114.866s
| Adam | epoch: 001 | loss: 21.98897 - acc: 0.0450 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 05824/19500
--
Training Step: 92  | total loss: 22.02070 | time: 116.132s
| Adam | epoch: 001 | loss: 22.02070 - acc: 0.0437 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 05888/19500
--
Training Step: 93  | total loss: 22.01328 | time: 117.398s
| Adam | epoch: 001 | loss: 22.01328 - acc: 0.0440 | val_loss: 22.10482 - va

Training Step: 131  | total loss: 21.61559 | time: 165.529s
| Adam | epoch: 001 | loss: 21.61559 - acc: 0.0612 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 08384/19500
--
Training Step: 132  | total loss: 21.64868 | time: 166.784s
| Adam | epoch: 001 | loss: 21.64868 - acc: 0.0598 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 08448/19500
--
Training Step: 133  | total loss: 21.67846 | time: 168.048s
| Adam | epoch: 001 | loss: 21.67846 - acc: 0.0585 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 08512/19500
--
Training Step: 134  | total loss: 21.63331 | time: 169.318s
| Adam | epoch: 001 | loss: 21.63331 - acc: 0.0605 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 08576/19500
--
Training Step: 135  | total loss: 21.52072 | time: 170.592s
| Adam | epoch: 001 | loss: 21.52072 - acc: 0.0654 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 08640/19500
--
Training Step: 136  | total loss: 21.49134 | time: 171.860s
| Adam | epoch: 001 | loss: 21.49134 - acc: 0.0666 | val_loss: 22.1048

Training Step: 174  | total loss: 21.71147 | time: 219.788s
| Adam | epoch: 001 | loss: 21.71147 - acc: 0.0571 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 11136/19500
--
Training Step: 175  | total loss: 21.80693 | time: 221.045s
| Adam | epoch: 001 | loss: 21.80693 - acc: 0.0529 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 11200/19500
--
Training Step: 176  | total loss: 21.71296 | time: 222.301s
| Adam | epoch: 001 | loss: 21.71296 - acc: 0.0570 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 11264/19500
--
Training Step: 177  | total loss: 21.77229 | time: 223.573s
| Adam | epoch: 001 | loss: 21.77229 - acc: 0.0544 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 11328/19500
--
Training Step: 178  | total loss: 21.64580 | time: 224.853s
| Adam | epoch: 001 | loss: 21.64580 - acc: 0.0599 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 11392/19500
--
Training Step: 179  | total loss: 21.56794 | time: 226.117s
| Adam | epoch: 001 | loss: 21.56794 - acc: 0.0633 | val_loss: 22.1048

...

Training Step: 520  | total loss: 21.66279 | time: 273.268s
| Adam | epoch: 002 | loss: 21.66279 - acc: 0.0592 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 13760/19500
--
Training Step: 521  | total loss: 21.58323 | time: 274.538s
| Adam | epoch: 002 | loss: 21.58323 - acc: 0.0627 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 13824/19500
--
Training Step: 522  | total loss: 21.54760 | time: 275.819s
| Adam | epoch: 002 | loss: 21.54760 - acc: 0.0642 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 13888/19500
--
Training Step: 523  | total loss: 21.58749 | time: 277.090s
| Adam | epoch: 002 | loss: 21.58749 - acc: 0.0625 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 13952/19500
--
Training Step: 524  | total loss: 21.51546 | time: 278.355s
| Adam | epoch: 002 | loss: 21.51546 - acc: 0.0656 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 14016/19500
--
Training Step: 525  | total loss: 21.45063 | time: 279.638s
| Adam | epoch: 002 | loss: 21.45063 - acc: 0.0684 | val_loss: 22.1048

Training Step: 563  | total loss: 21.68242 | time: 328.093s
| Adam | epoch: 002 | loss: 21.68242 - acc: 0.0583 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 16512/19500
--
Training Step: 564  | total loss: 21.70883 | time: 329.361s
| Adam | epoch: 002 | loss: 21.70883 - acc: 0.0572 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 16576/19500
--
Training Step: 565  | total loss: 21.80456 | time: 330.629s
| Adam | epoch: 002 | loss: 21.80456 - acc: 0.0530 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 16640/19500
--
Training Step: 566  | total loss: 21.78278 | time: 331.924s
| Adam | epoch: 002 | loss: 21.78278 - acc: 0.0540 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 16704/19500
--
Training Step: 567  | total loss: 21.72719 | time: 333.188s
| Adam | epoch: 002 | loss: 21.72719 - acc: 0.0564 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 16768/19500
--
Training Step: 568  | total loss: 21.74913 | time: 334.469s
| Adam | epoch: 002 | loss: 21.74913 - acc: 0.0554 | val_loss: 22.1048

Training Step: 606  | total loss: 21.91300 | time: 382.961s
| Adam | epoch: 002 | loss: 21.91300 - acc: 0.0483 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 19264/19500
--
Training Step: 607  | total loss: 21.98831 | time: 384.243s
| Adam | epoch: 002 | loss: 21.98831 - acc: 0.0451 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 19328/19500
--
Training Step: 608  | total loss: 21.76826 | time: 385.500s
| Adam | epoch: 002 | loss: 21.76826 - acc: 0.0546 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 19392/19500
--
Training Step: 609  | total loss: 21.75011 | time: 386.765s
| Adam | epoch: 002 | loss: 21.75011 - acc: 0.0554 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 19456/19500
--
Training Step: 610  | total loss: 21.73377 | time: 388.031s
| Adam | epoch: 002 | loss: 21.73377 - acc: 0.0561 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 19500/19500
--
Training Step: 611  | total loss: 21.64711 | time: 1.209s
| Adam | epoch: 003 | loss: 21.64711 - acc: 0.0599 | val_loss: 22.10482 

Training Step: 650  | total loss: 21.69084 | time: 50.482s
| Adam | epoch: 003 | loss: 21.69084 - acc: 0.0580 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 02560/19500
--
Training Step: 651  | total loss: 21.75239 | time: 51.739s
| Adam | epoch: 003 | loss: 21.75239 - acc: 0.0553 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 02624/19500
--
Training Step: 652  | total loss: 21.77180 | time: 53.001s
| Adam | epoch: 003 | loss: 21.77180 - acc: 0.0545 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 02688/19500
--
Training Step: 653  | total loss: 21.82525 | time: 54.254s
| Adam | epoch: 003 | loss: 21.82525 - acc: 0.0521 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 02752/19500
--
Training Step: 654  | total loss: 21.90933 | time: 55.522s
| Adam | epoch: 003 | loss: 21.90933 - acc: 0.0485 | val_loss: 22.10482 - val_acc: 0.0400 -- iter: 02816/19500
--
Training Step: 655  | total loss: 21.84109 | time: 56.779s
| Adam | epoch: 003 | loss: 21.84109 - acc: 0.0515 | val_loss: 22.10482 - va

KeyboardInterrupt: 